In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
path = '/kaggle/input/titanic/train.csv'
titanic_data_train = pd.read_csv(path)

In [ ]:
path_test = '/kaggle/input/titanic/test.csv'
titanic_data_test = pd.read_csv(path_test)

In [ ]:
titanic_data_train

In [ ]:
print('test data')
titanic_data_test.head()

In [ ]:
path_gender = '/kaggle/input/titanic/gender_submission.csv'
data_titanic_gender = pd.read_csv(path_gender)
data_titanic_gender.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
sb.scatterplot(titanic_data_train['Age'],titanic_data_train['Fare'],hue = titanic_data_train['Survived'])

In [ ]:
plt.figure(figsize=(35,10))
sb.countplot(x=titanic_data_train['Age'], data=titanic_data_train['Fare'])
plt.show()

In [ ]:
sb.lineplot(x=titanic_data_train['Age'],y=titanic_data_train['Fare'])

**we will train a model that predicts whether a person will survive based on his age and gender.**

In [ ]:
titanic_data_train.info()

In [ ]:
# droping all unecessary cells in order training our data
del(titanic_data_train['PassengerId'])
del(titanic_data_train['Name'])
del(titanic_data_train['Ticket'])
del(titanic_data_train['Cabin'])

In [ ]:
titanic_data_train.info()

In [ ]:
# this is to replace each value of cell with it's own integer
titanic_data_train['Sex'] = titanic_data_train['Sex'].replace({'male':0 , 'female':1})
titanic_data_train['Embarked'] = titanic_data_train['Embarked'].replace({'C':0, 'Q':1, 'S':2})

In [ ]:
# this is to replace all nan values into a zero
titanic_data_train['Age'] = titanic_data_train['Age'].fillna(0)
titanic_data_train.head(20)

In [ ]:
# this will drop all nan values (you can test without it)
titanic_data_train = titanic_data_train.dropna()

In [ ]:
# one hot encoding where we make each cell have it's own values

le=LabelEncoder()

titanic_data_train['Survived']=le.fit_transform(titanic_data_train['Survived'])
titanic_data_train['Sex']=le.fit_transform(titanic_data_train['Sex'])
titanic_data_train['Embarked']=le.fit_transform(titanic_data_train['Embarked'])

In [ ]:
y = titanic_data_train['Survived']
x = titanic_data_train.drop(['Survived'],axis=1)
print(x,y)

In [ ]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(x, y ,test_size=0.2, random_state=30)

***Using randomForest :***

In [ ]:
randomF = RandomForestClassifier()
randomF.fit(train_X,train_y)

In [ ]:
pred_RF = randomF.predict(val_X)

In [ ]:
score_RF = accuracy_score(val_y,pred_RF)
score_RF

In [ ]:
print("Accuracy:",metrics.accuracy_score(val_y,pred_RF))
print("Precision:",metrics.precision_score(val_y,pred_RF))
print("Recall:",metrics.recall_score(val_y,pred_RF))

***Using logisticRegression :***

In [ ]:
log = LogisticRegression(solver='liblinear')
log.fit(train_X, train_y)

In [ ]:
pred_log = log.predict(val_X)

In [ ]:
score_log = accuracy_score(val_y,pred_log)
score_log

In [ ]:
print("Accuracy:",metrics.accuracy_score(val_y,pred_log))
print("Precision:",metrics.precision_score(val_y,pred_log))
print("Recall:",metrics.recall_score(val_y,pred_log))

***Using XGBoost :***

In [ ]:
xgb = xgb.XGBClassifier(objectives='binary:logistic',n_estimators=10)
xgb.fit(train_X , train_y)

In [ ]:
pred_xgb = xgb.predict(val_X)

In [ ]:
score_xgb = accuracy_score(val_y,pred_xgb)
score_xgb

In [ ]:
print("Accuracy:",metrics.accuracy_score(val_y,pred_xgb))
print("Precision:",metrics.precision_score(val_y,pred_xgb))
print("Recall:",metrics.recall_score(val_y,pred_xgb))

**we notice that XGBoost has the highest accuracy between the 3 models we will use it for submitting**

In [ ]:
del(titanic_data_test['PassengerId'])
del(titanic_data_test['Name'])
del(titanic_data_test['Ticket'])
del(titanic_data_test['Cabin'])

In [ ]:
# this is to replace each value of cell with it's own integer
titanic_data_test['Sex'] = titanic_data_test['Sex'].replace({'male':0 , 'female':1})
titanic_data_test['Embarked'] = titanic_data_test['Embarked'].replace({'C':0, 'Q':1, 'S':2})
titanic_data_test['Age'] = titanic_data_test['Age'].fillna(0)

In [ ]:
X_test = titanic_data_test.copy()

In [ ]:
pred = xgb.predict(X_test)

In [ ]:
data_titanic_gender['Survived'] = pred
data_titanic_gender.to_csv('submission.csv',index=False)